Продакт-менеджер Василий попросил вас проанализировать завершенные уроки и ответить на следующие вопросы:

1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене) (7 баллов).

2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*. (5 баллов)

3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). (5 баллов) 

4. Выяви самые популярные предметы (ТОП-3) по количеству регистраций на них. А также предметы с самым большим оттоком (ТОП-3). (8 баллов)

5. Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. Описание подхода можно найти тут.(10 баллов) 

6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 0.1 до 0.5, monetary от 55 до 72 баллов. Описание подхода можно найти тут. (35 баллов)

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен

**Загрузка начальных данных, используемых библиотек. Просмотр значений**

In [190]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import urllib
import json
from urllib.parse import urlencode

In [191]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/PBW7aUHGuodFDA'

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
df = pd.read_csv(download_url)
df.head(4)

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0


In [192]:
na_number = df.isna().sum()
na_number

code_module           0
code_presentation     0
id_assessment         0
assessment_type       0
date                 11
weight                0
dtype: int64

In [193]:
df_types = df.dtypes
df_types

code_module           object
code_presentation     object
id_assessment          int64
assessment_type       object
date                 float64
weight               float64
dtype: object

In [194]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/m0Z6QYNT46f9tQ'


final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
df2 = pd.read_csv(download_url)
df2.head(4)

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262


In [195]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/lsmdbYB0iM7p3w'

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
df3 = pd.read_csv(download_url)
df3.head(4)

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0


In [196]:
df3.id_student.nunique()

23369

**Задание. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене)**

*Соединяем таблицу с данными об оценках в тестах с таблицей результатов оценок студентов.
По условию задания требуется отобрать только результаты по экзаменам. Дополнительно исключаем оценки студентов, получивших перезачет.*

In [197]:
df_assess_stud = df.merge(df3, on = 'id_assessment')
df_assess_stud = df_assess_stud.query('is_banked == 0 and assessment_type == "Exam"')
df_assess_stud.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
52923,CCC,2014B,24290,Exam,NaN,100.0,558914,230,0,32.0
52924,CCC,2014B,24290,Exam,NaN,100.0,559706,234,0,78.0
52925,CCC,2014B,24290,Exam,NaN,100.0,559770,230,0,54.0
52926,CCC,2014B,24290,Exam,NaN,100.0,560114,230,0,64.0
52927,CCC,2014B,24290,Exam,NaN,100.0,560311,234,0,100.0


*Отбираем данные только по успешной сдаче. По условию успешная сдача - результат >= 40 баллов.*

In [198]:
df_assess_stud_exam_succ = df_assess_stud.query("score >= 40")
df_assess_stud_exam_succ.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
52924,CCC,2014B,24290,Exam,NaN,100.0,559706,234,0,78.0
52925,CCC,2014B,24290,Exam,NaN,100.0,559770,230,0,54.0
52926,CCC,2014B,24290,Exam,NaN,100.0,560114,230,0,64.0
52927,CCC,2014B,24290,Exam,NaN,100.0,560311,234,0,100.0
52928,CCC,2014B,24290,Exam,NaN,100.0,560494,230,0,92.0


*Считаем количество студентов, которые сдали успешно (сдали экзамен) только по 1 курсу.*

In [199]:
df_assess_stud_exam_succ.groupby('id_student', as_index = False).agg({'code_module':'nunique'}).query('code_module == 1').shape[0]

3805

**ОТВЕТ: Количество студентов, которые успешно сдали только один курс составляет 3805 человек.**

**ЗАДАНИЕ. Выяви самый сложный и самый простой экзамен: 
найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью.
завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен**

*Создадим столбец с показателем успешности сдачи экзамена студентом.*

In [200]:
df_assess_stud['success'] = np.where(df_assess_stud['score'] > 40, 1, 0)
df_assess_stud.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,success
52923,CCC,2014B,24290,Exam,NaN,100.0,558914,230,0,32.0,0
52924,CCC,2014B,24290,Exam,NaN,100.0,559706,234,0,78.0,1
52925,CCC,2014B,24290,Exam,NaN,100.0,559770,230,0,54.0,1
52926,CCC,2014B,24290,Exam,NaN,100.0,560114,230,0,64.0,1
52927,CCC,2014B,24290,Exam,NaN,100.0,560311,234,0,100.0,1


*Рассчитаем количество успешных и неуспешных попыток сдачи экзамена для каждого id курса.*

In [201]:
df_success_table = df_assess_stud.groupby(['id_assessment', 'success', 'code_module'], as_index = False).agg({'score' : 'count'})
df_success_table

,id_assessment,success,code_module,score
0,24290,0,CCC,97
1,24290,1,CCC,650
2,24299,0,CCC,173
3,24299,1,CCC,995
4,25340,0,DDD,124
5,25340,1,DDD,478
6,25354,0,DDD,112
7,25354,1,DDD,856
8,25361,0,DDD,46
9,25361,1,DDD,478


In [202]:
df_success_table_all_attempts = df_assess_stud.groupby(['id_assessment'], as_index = False).agg({'score' : 'count'})
df_success_table_all_attempts

,id_assessment,score
0,24290,747
1,24299,1168
2,25340,602
3,25354,968
4,25361,524
5,25368,950


*Дополним таблицу общими попытками сдачи и покажем число только успешных попыток (чтобы далее рассчитать долю успешной сдачи курсов)*

In [203]:
df_final_attempts = df_success_table_all_attempts.merge(df_success_table, on = 'id_assessment')
df_final_attempts = df_final_attempts[df_final_attempts['success'] == 1]
df_final_attempts = df_final_attempts.rename(columns = {'score_x' : 'all_atempts', 'score_y' : 'succ_attempts'})
df_final_attempts

,id_assessment,all_atempts,success,code_module,succ_attempts
1,24290,747,1,CCC,650
3,24299,1168,1,CCC,995
5,25340,602,1,DDD,478
7,25354,968,1,DDD,856
9,25361,524,1,DDD,478
11,25368,950,1,DDD,825


*Находим долю успешной сдачи курсов. Чем она меньше, тем сложнее курс. По таблице определяем ответ к заданию*

In [204]:
df_final_attempts['success_rate'] = df_final_attempts['succ_attempts']/df_final_attempts['all_atempts']
df_final_attempts = df_final_attempts[['id_assessment', 'code_module', 'success_rate']]
df_final_attempts.sort_values(by='success_rate')

,id_assessment,code_module,success_rate
5,25340,DDD,0.794020
3,24299,CCC,0.851884
11,25368,DDD,0.868421
1,24290,CCC,0.870147
7,25354,DDD,0.884298
9,25361,DDD,0.912214


*Соответственно, курс с id_assessment=25340, модуль DDD - самый сложный, с id_assessment=25361 модуль DDD - самый легкий*

**ОТВЕТ: Самый сложный курс: id_assessment=25340 модуль DDD, самый простой курс - id_assessment=25361 модуль DDD.**

**ЗАДАНИЕ: По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом)**

*Отбираем только успешные попытки сдачи экзаменов*

In [205]:
df_assess_stud_exam_success = df_assess_stud[df_assess_stud['success'] == 1]
df_assess_stud_exam_success

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score,success
52924,CCC,2014B,24290,Exam,NaN,100.0,559706,234,0,78.0,1
52925,CCC,2014B,24290,Exam,NaN,100.0,559770,230,0,54.0,1
52926,CCC,2014B,24290,Exam,NaN,100.0,560114,230,0,64.0,1
52927,CCC,2014B,24290,Exam,NaN,100.0,560311,234,0,100.0,1
52928,CCC,2014B,24290,Exam,NaN,100.0,560494,230,0,92.0,1
...,...,...,...,...,...,...,...,...,...,...,...
95978,DDD,2014J,25368,Exam,NaN,100.0,652617,243,0,56.0,1
95980,DDD,2014J,25368,Exam,NaN,100.0,652680,243,0,91.0,1
95981,DDD,2014J,25368,Exam,NaN,100.0,652732,249,0,47.0,1
95982,DDD,2014J,25368,Exam,NaN,100.0,652965,242,0,87.0,1


*Группируем данные по id курса и находим ответ к заданию.*

In [206]:
df_assess_stud_exam_success_date = df_assess_stud_exam_success.groupby(['code_module'], as_index = False).agg({'date_submitted' : 'mean'})
df_assess_stud_exam_success_date

,code_module,date_submitted
0,CCC,239.341033
1,DDD,238.020478


In [207]:
mean_date_submitted_CCC = df_assess_stud_exam_success_date.iloc[0][1]
mean_date_submitted_DDD = df_assess_stud_exam_success_date.iloc[1][1]
mean_date_submitted_CCC, mean_date_submitted_DDD

(239.34103343465046, 238.02047781569965)

**ОТВЕТ: Средний срок сдачи составляет: курс ССС: 239.4 дня с начала семестра, курс DDD 238 дней с начала семестра.**

**ЗАДАНИЕ. Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3)**

*Рассмотрим таблицу с данными о регистрации студентов*

In [208]:
base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/Yse4Y6RJqg_WaA'

final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
df4 = pd.read_csv(download_url)
df4.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [209]:
na_number = df4.isna().sum()
na_number

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

*Сгруппируем данные по названию курса и найдем количество записавшихся на каждый из курсов студентов.
Далее определяем ТОП-3 самых популярных курсов по количеству регистраций и ТОП-3 по количеству отписок.*

In [210]:
df_registration = df4.groupby(['code_module'], as_index = False).agg({'date_registration' : 'count', 'date_unregistration' : 'count'})
df_registration

,code_module,date_registration,date_unregistration
0,AAA,748,126
1,BBB,7900,2377
2,CCC,4426,1947
3,DDD,6257,2235
4,EEE,2932,718
5,FFF,7751,2380
6,GGG,2534,289


In [211]:
Top3_most_favourite = df_registration.sort_values(by = 'date_registration', ascending = False).head(3)
Top3_most_favourite

,code_module,date_registration,date_unregistration
1,BBB,7900,2377
5,FFF,7751,2380
3,DDD,6257,2235


*Строим столбцец, где 1 - студент не отменял регистрацию, 0 - студент отменил регистрацию*

In [212]:
Top3_most_hated = df_registration.sort_values(by = 'date_unregistration', ascending = False).head(3)
Top3_most_hated

,code_module,date_registration,date_unregistration
5,FFF,7751,2380
1,BBB,7900,2377
3,DDD,6257,2235


**ОТВЕТ: ТОП-3 Популярных курса: BBB, FFF, DDD. ТОП-3 курса по оттоку студентов: FFF, BBB, DDD.**

**ЗАДАНИЕ: Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.**

*Строим когорты на основе объеденных таблиц с оценками по курсам и таблиц с оценками студентов. Проводим вычисления, аналогичные проведенным выше. Объединяем полученные ДФ с завершаемостью курсов и средним сроком сдачи в один ДФ и возвращаем*

In [213]:
 def Cohorts():
    base_df = df.merge(df3, on='id_assessment')
    df_exam_only=base_df.query('assessment_type == "Exam"')
    df_exam_allstudents=df_exam_only.groupby(['code_presentation'],as_index=False).\
    agg(count_students = ('id_student', 'nunique'))
    
    df_exam_passed= df_exam_only[df_exam_only.score >= 40].groupby(['code_presentation'],as_index=False) \
                                   .agg(count_passed_students = ('id_student', 'nunique'))
    
    df_new_compl_rate = df_exam_allstudents.merge(df_exam_passed, on = 'code_presentation')
    df_new_compl_rate['completion_rate'] = (df_new_compl_rate.count_passed_students / df_new_compl_rate.count_students).round(2)
    df_new_compl_rate = df_new_compl_rate.pivot_table(index='code_presentation', values='completion_rate')
    
    df_new_dates = df_exam_only.groupby(['code_presentation'],as_index = False).agg({'date_submitted':'mean'})
    df_new_dates = df_new_dates.rename(columns = {'date_submitted' : 'mean_date_submitted'})
    
    final_df = df_new_dates.merge(df_new_compl_rate, on = 'code_presentation')
    
    return final_df

In [214]:
Cohorts()

,code_presentation,mean_date_submitted,completion_rate
0,2013B,230.179402,0.84
1,2013J,239.505165,0.91
2,2014B,232.988985,0.90
3,2014J,243.674221,0.88


**ОТВЕТ: 2013B - семестр с самой низкой завершаемостью, 2014J - семестр с самым долгим средним сроком сдачи курса.**	

**ЗАДАНИЕ: Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию.**

*Соединим таблицу assessments и students assessments.*

In [215]:
df_study = df.merge(df3, on='id_assessment')
df_study.head(3)

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
0,AAA,2013J,1752,TMA,19.0,10.0,11391,18,0,78.0
1,AAA,2013J,1752,TMA,19.0,10.0,28400,22,0,70.0
2,AAA,2013J,1752,TMA,19.0,10.0,31604,17,0,72.0


*Из нашей общей таблицы df1 возьмем только результаты экзаменов*

In [216]:
df_RFM=df_study.query('assessment_type == "Exam"')
df_RFM.head(4)

,code_module,code_presentation,id_assessment,assessment_type,date,weight,id_student,date_submitted,is_banked,score
52923,CCC,2014B,24290,Exam,NaN,100.0,558914,230,0,32.0
52924,CCC,2014B,24290,Exam,NaN,100.0,559706,234,0,78.0
52925,CCC,2014B,24290,Exam,NaN,100.0,559770,230,0,54.0
52926,CCC,2014B,24290,Exam,NaN,100.0,560114,230,0,64.0


Таким образом из 23369 студентов экзамен сдавали 4633, что приблизительно равно 20%

In [217]:
df3.id_student.nunique(), df_RFM.id_student.nunique()

(23369, 4633)

*Построим RFM кластеры для 20% студентов, которые сдавали экзамены, с учетом того, что 80% студентов еще не приступали к сдаче экзаменов.*

Кластер R - среднее время сдачи одного экзамена

In [218]:
df_overall=df_study.merge(df2, on=['code_module', 'code_presentation'])

*Подготовим запрос для расчета когорт R* 

In [219]:
df_RFM_R=df_overall.query('assessment_type=="Exam"').groupby(['id_student', 'id_assessment']).\
agg({'date_submitted':'mean', 'module_presentation_length':'mean', 'score':'mean'}).reset_index()
df_RFM_R

,id_student,id_assessment,date_submitted,module_presentation_length,score
0,23698,24299,243,269,80.0
1,24213,25361,236,241,58.0
2,27116,24299,243,269,96.0
3,28046,25354,237,261,40.0
4,28787,24299,243,269,44.0
...,...,...,...,...,...
4954,2694886,25361,236,241,69.0
4955,2694933,25340,230,240,73.0
4956,2695608,25354,237,261,73.0
4957,2697181,24290,230,241,80.0


3 когорта R, это те, кто сдал экзамен после окончания семестра 

In [220]:
df_RFM_passed_later=df_RFM_R.query('date_submitted>module_presentation_length')
df_RFM_passed_later['R']=3
df_RFM_passed_later.shape

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(12, 6)

1 когорта R, это те, кто сдал экзамен в срок

In [221]:
df_RFM_passed_on_time=df_RFM_R.query('date_submitted<module_presentation_length')
df_RFM_passed_on_time['R']=1
df_RFM_passed_on_time.shape

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(4947, 6)

In [222]:
df_RFM_R=pd.concat([df_RFM_passed_later, df_RFM_passed_on_time])
df_RFM_R_final=df_RFM_R.groupby('id_student').agg({'R':'mean'}).reset_index()
df_RFM_R_final[df_RFM_R_final['R']==3].shape, df_RFM_R_final[df_RFM_R_final['R']==1].shape

((12, 2), (4621, 2))

M - среднее кол-во баллов за экзамен

Примем следующие когорты, соответствущие оценкам:
- < 40 - "неудовлетворительно"
- 40 - 69 - "удовлетворительно"
- 70 - 84 - "хорошо"
- от 85 - "отлично"

In [223]:
def m_score(x):
    if  x < 40:
        return 1
    elif 40<= x <= 69:
        return 2
    elif 70<= x < 85:
        return 3
    else:
        return 4

выборка для разбивки на когорты

In [224]:
df_M=df_RFM.groupby('id_student').agg({'score':'mean'}).reset_index()
df_M     

,id_student,score
0,23698,80.0
1,24213,58.0
2,27116,96.0
3,28046,40.0
4,28787,44.0
...,...,...
4628,2694886,69.0
4629,2694933,73.0
4630,2695608,73.0
4631,2697181,80.0


In [225]:
df_M['M']=df_M.score.apply(lambda x: m_score(x))
df_M

,id_student,score,M
0,23698,80.0,3
1,24213,58.0,2
2,27116,96.0,4
3,28046,40.0,2
4,28787,44.0,2
...,...,...,...
4628,2694886,69.0,2
4629,2694933,73.0,3
4630,2695608,73.0,3
4631,2697181,80.0,3


F - завершаемость курсов

Формируем запрос для последующего построения когорт. Сначала учтем тех, кто завершили курс.

In [226]:
df_RFM_F_completed = df_study.query('assessment_type == "Exam" and score >= 40')\
 .groupby(['id_student', 'id_assessment'], as_index=False).agg({'assessment_type': 'count'})\
 .rename(columns={"assessment_type": "F"})\
 .groupby('id_student').agg({'F':'sum'}).reset_index()
df_RFM_F_completed 

,id_student,F
0,23698,1
1,24213,1
2,27116,1
3,28046,1
4,28787,1
...,...,...
4092,2694886,1
4093,2694933,1
4094,2695608,1
4095,2697181,1


Не завершили курс

In [227]:
df_RFM_F_failed  = df_study.query('assessment_type == "Exam" and score < 40')\
 .groupby(['id_student', 'id_assessment'],as_index=False).agg({'assessment_type': 'count'})\
 .rename(columns={"assessment_type": "F"})\
 .groupby('id_student').agg({'F':'sum'}).reset_index()

df_RFM_F_failed.F=0
df_RFM_F_failed

,id_student,F
0,31173,0
1,46753,0
2,51955,0
3,52014,0
4,63566,0
...,...,...
559,2611779,0
560,2615956,0
561,2655683,0
562,2661870,0


Соединяем две выборки. Проверяем, что не потеряли данные

In [228]:
df_RFM_F = pd.merge(df_RFM_F_completed, df_RFM_F_failed, on=['id_student', 'F'], how='outer')
df_RFM_F.id_student.nunique()

4633

In [229]:
def F_cohort(y):
    if y==2.0:
        return 4
    elif y==1.0:
        return 3
    elif y==0.5:
        return 2
    else:
        return 1

In [230]:
df_RFM_F=df_RFM_F.groupby('id_student', as_index=False).agg({'F':'mean'})
df_RFM_F.F=df_RFM_F.F.apply(lambda y: F_cohort(y))
df_RFM_F

,id_student,F
0,23698,3
1,24213,3
2,27116,3
3,28046,3
4,28787,3
...,...,...
4628,2694886,3
4629,2694933,3
4630,2695608,3
4631,2697181,3


Соединяем три когорты

In [231]:
RFM_RM= df_RFM_R_final.merge(df_M, on='id_student')
RFM = RFM_RM.merge(df_RFM_F, on='id_student')
RFM.drop(columns=['score'])

,id_student,R,M,F
0,23698,1,3,3
1,24213,1,2,3
2,27116,1,4,3
3,28046,1,2,3
4,28787,1,2,3
...,...,...,...,...
4628,2694886,1,2,3
4629,2694933,1,3,3
4630,2695608,1,3,3
4631,2697181,1,3,3


R - среднее время сдачи одного экзамена
1 - все экзамены сданы до начала семестра;  
2 - один экзамен до окончания семестра, один экзамен после;
3 - сдавал все свои экзамены (один или два) после окончания семестра.

F - завершаемость курсов
1 - курсов завершено: 0;
2 - завершено 1 из 2-х курсов;
3 - завершено 1 из 1 курса;
4 - завершено 2 из 2-х курсов.

M - среднее количество баллов, получаемое за экзамен
Баллы имеют конкретное знание для оценки знаний и дальнейшего обучения, поэтому мы разделим на такие ранги: 
1 - < 40 - "неудовлетворительно";
2 - 40 - 69 - "удовлетворительно";
3 - 70 - 84 - "хорошо";
4 - от 85 - "отлично".

**ОТВЕТ: На основании rfm разбивки разбивки каждому студенту присвоен номер, по которому, можно определить основные характеристики каждого студента**

In [232]:
RFM['RFM'] = RFM['R'].apply(str) + RFM['F'].apply(str) + RFM['M'].apply(str) 

RFM.RFM.value_counts().to_frame()

,RFM
132,1946
133,1001
134,817
111,534
142,109
144,96
143,90
122,21
121,7
332,7
